## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: `<почта> - <дата утечки> - <источник утечки> - <описание утечки>`  

**Подсказка**: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

### Дополнительная часть (необязательная)


Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)  
```
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ  
```

В итоге должен формироваться датафрейм со столбцами: `<дата поста> - <текст поста>`

In [1]:
import requests
import pandas
import numpy

In [2]:
def get_avast_response(emails: [str]) -> requests.Response:
    return requests.post(
        url='https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data',
        json={'emailAddresses': emails},
        headers={
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15',
            'Vaar-Version': '0',
            'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
            'Vaar-Header-App-Build-Version': '1.0.0'}
        )

avast_response = get_avast_response(
    emails=[
        'oradrim@mail.ru',
        'bikhev@yandex.ru'
    ]
)

In [3]:
def transform_to_data_frame(leaks: dict) -> pandas.DataFrame:
    data_frame = pandas.DataFrame()
    summary_key, breaches_key = [
        'summary', 'breaches'
    ]

    # Checking for the presence of data by keys
    for key in [summary_key, breaches_key]:
        if not key in leaks:
            return data_frame

    # Iterate through all available emails
    for email_key, values in leaks[summary_key].items():
        if not breaches_key in values:
            continue

        # Iterate every leak in the current mail (email_key)
        for branch in values[breaches_key]:
            line = get_line_data_frame(
                leaks=leaks,
                branch_key=str(branch),
                email=email_key
            )
            data_frame = pandas.concat([data_frame, line])

    return  data_frame

def get_line_data_frame(leaks: dict, branch_key: str, email: str) -> pandas.DataFrame:
    breaches_key, publishDate_key, description_key, site_key = [
        'breaches', 'publishDate', 'description', 'site'
    ]
    breaches = leaks[breaches_key]

    # Checking for the presence of data by keys
    if not branch_key in breaches:
        return pandas.DataFrame()

    branch = breaches[branch_key]

    # Checking for the presence of data by keys
    for key in [site_key, publishDate_key, description_key]:
        if not key in branch:
            return pandas.DataFrame()

    return pandas.DataFrame({
        'email': [email],
        'date': [branch[publishDate_key]],
        'source': [branch[site_key]],
        'description': [branch[description_key]]
    })

leaks_table = transform_to_data_frame(
    leaks=avast_response.json()
)

leaks_table


,email,date,source,description
0,oradrim@mail.ru,2019-03-20T00:00:00Z,bookmate.com,"In July 2018, Bookmate was allegedly breached...."
0,oradrim@mail.ru,2020-05-21T00:00:00Z,vk.com,"At some time in 2020, the Russian social netwo..."
0,oradrim@mail.ru,2016-10-21T00:00:00Z,myspace.com,"Shortly before the 2016 Memorial Day weekend, ..."
0,oradrim@mail.ru,2016-11-01T00:00:00Z,qip.ru,"In 2011, Russian instant messaging service pro..."
0,oradrim@mail.ru,2017-06-14T00:00:00Z,nnm-club.me,"In September 2013, Russian torrent tracker NNM..."
0,bikhev@yandex.ru,2016-10-26T00:00:00Z,twitter.com,Login credentials for over 32 Million Twitter ...
